# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123229e+02     1.539109e+00
 * time: 0.08958196640014648
     1     1.112763e+01     9.443360e-01
 * time: 1.0209548473358154
     2    -1.149957e+01     1.018354e+00
 * time: 1.152420997619629
     3    -3.376798e+01     7.366869e-01
 * time: 1.3051729202270508
     4    -4.745915e+01     5.694787e-01
 * time: 1.4545609951019287
     5    -5.664495e+01     1.965651e-01
 * time: 1.6011438369750977
     6    -5.959904e+01     1.365357e-01
 * time: 1.7216949462890625
     7    -6.081899e+01     5.975411e-02
 * time: 1.8292210102081299
     8    -6.131186e+01     7.635193e-02
 * time: 1.9471349716186523
     9    -6.163622e+01     5.074887e-02
 * time: 2.059048891067505
    10    -6.187253e+01     2.594362e-02
 * time: 2.165742874145508
    11    -6.203107e+01     2.030140e-02
 * time: 2.2800488471984863
    12    -6.209444e+01     1.716454e-02
 * time: 2.3922719955444336
    13    -6.215533e+01     1.322476e-02
 * time: 2.506537914276

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557679
    AtomicNonlocal      14.8522646
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666460797
